In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections

from pocr import PostOcr, showDistribution, showDistributionFreq

Load the corpus

In [29]:
P = PostOcr(4)

Loading TF data ...
Loading tf data from ~/github/annotation/clariah-dr/tf/daghregister/004/0.1
done
24788 words in 229023 occurrences
All words frequencies written to ~/github/annotation/clariah-dr/postocr/daghregister/004/wordfreqs.tsv
word frequency│word occurrence
──────────────┼─────────────────────────────────────────────────────────────────────────────────────
          7714│■1
          5392│■1
          3770│■■■3
          3152│■1
          2635│■1
          2203│■■2
          1540│■■■■■■6
          1288│■■■3
          1077│■■■■■■6
           900│■1
           753│■■■■■■■■■9
           629│■■■■■■■7
           526│■■■■■■■■■■■■13
           440│■■■■■■■■■■■■■■17
           368│■■■■■■■■■9
           308│■■■■■■■■■■■■■■■■■■26
           257│■■■■■■■■■■■■■■■■21
           215│■■■■■■■■■■■■■■■18
           180│■■■■■■■■■■■■■■■■■25
           150│■■■■■■■■■■■■■■■■■■28
           126│■■■■■■■■■■■■■■■■■■■■■40
           105│■■■■■■■■■■■■■■■■■■■■■■■48
            88│■■■■■■■■■■■■■■■■■■■■■■■47
  

# Morpheme splitting by information reduction

Idea: pick a word and try to split off morpheme-like bits from the start and from the end of a word.
We do it in rounds, in each round at most one morf per side of the word is split off.

What we do is: 

```
word = morf + main
```

and/or

```
word = main + morf
```

How do we determine whether a possible split is a good split?
By using several indicators:

1. the number of other *mains* that can combined with the *morf*
2. the number of other *morfs* that can be combined with the *main*
3. the quality of those other *mains*
4. the quality of those other *morfs*.

We then weigh the values found for these indicators, and take a decision.

When we look for possible *morfs* that combine with a *main*, we also want to know
if the *main* with the empty *morf* is a valid word.
Likewise, when we look for possible *mains* that combine with a *morf*,
we take the empty *main* into account.

We use an upper bound for morf lengths.

In short words, if we detect overlapping morfs at opposing ends, we pick the one with the best indicators

# Preparation

We compute data in advance that we will need frequently: the compositions of all word forms.

In [4]:
P.getCompositions()

82115 distinct suffixes
85590 distinct prefixes


# Quality of a word split

The computation of the quality measures is potentially costly.
We do not compute them in advance for the whole corpus, but on a just-in-time basis.
Whenever we have computed a quality measure, we memoize it, so that it can be retrieved rather than
computed when it is needed again.

This pays off, because the quality of an item is typically computed by applying a formula to
the qualities of a (big) number of other items.

We experiment with notions of quality.

Suppose we have `word = morf + main`.

A simple notion of quality is the amount of possible *mains* that are possible after the *morf*.

But this will give an enormous quality to one-letter morfs. In general, there is a big number of words starting with the
same letter.
A bit more sophisticated is to weigh all *mains*, by counting how many other *morfs* can precede it.
If `mainOther` is another main that can follow `morf`, but `mainOther` cannot be preceded by any other *morf* than `morf`,
it does not contribute to the quality of `morf` as morpheme.
But if it can be preceded by multiple *morfs*, it contributes to the fact that `morf` is a morpheme.

So the quality of a *morf* is the weighted sum of all its (non-empty) *mains* with which it forms a word,
where each main is weighted by the number of other *morfs* with which it can form a word, *including* the empty *morf*.

The quality of a *main* is the non-weighted sum of all its other (possibly empty) *morfs* with which it forms a word.

**Note the asymmetry:**

In the quality of a *morph*, we weigh its *mains* in such a way that a *main* that is a word on its own does contribute.

In the quality of a *main*, we weigh its *morfs* in such a way that a *morf* that is a word on its own does *not* contribute.

In the quality of a *morf*, we do weigh the possible mains.

In the quality of a *main*, we just take the number of other morfs, not the sum of their qualities.
If a *main* has exactly one morf, then the result is 0; such mains are not good mains.

How how do we weigh an empty morf exactly? If we do not say anything, it will be the sum of all words in the corpus.
Instead, the empty morf has weight 1.

**Recursive closure?**

Note that we could also have defined the notion of quality in a recursive way:

The quality of a *morf* is the sum of the qualities of its *mains*.

The quality of a *main* is the sum of the qualities of its *morfs*.

The question is then: how does this recursion stop?

We can approach this definition by the one we have given, by computing the qualities in rounds, where in each round we
compute new qualities based on the old qualities.
And then we stop when the qualities do not change anymore.

But at present I have no idea whether this will ever stop.

So we stick with just one iteration.

# Building intuition

We show first these quality measures in some examples, in order to get intuition.
When we have seen enough examples, we can proceed to define a decision procedure
based on the quality measures.

Some examples.

In [5]:
EXAMPLES = """
aanneemt
neemt
aengeschreeven
H
hunner
monterende
gelegentheyt
ongelegentheyt
heeft
gesanten
ootmoedige
""".strip().split()

In [7]:
P.COMPOSE[True]["eemt"]

{'aann', 'n', 'voorn', 'vr'}

In [22]:
P.clearQuality()
len(P.getFills("aann", "eemt", False, False))

0

In [17]:
P.getQuality("vr", "eemt", False, False)

75

Examine the examples:

In [28]:
len(P.COMPOSE[False]["n"])

414

In [27]:
len(P.getFills("n", None, True, False))

414

In [24]:
P.clearQuality()
P.showExamples(EXAMPLES)

**`aanneemt`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | a | anneemt | 9435 | 0 | 0 | 1114
2 | aa | nneemt | 1304 | 0 | 0 | 5
3 | aan | neemt | 23 | 2 | 2 | 3
4 | aann | eemt | 3 | 3 | 3 | 0
5 | aanne | emt | 14 | 14 | 14 | 0

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
7 | aanneem | t | 9380 | 0 | 0 | 3470
6 | aannee | mt | 653 | 0 | 0 | 47
5 | aanne | emt | 298 | 0 | 0 | 22
4 | aann | eemt | 245 | 0 | 0 | 3
3 | aan | neemt | 80 | 1 | 1 | 1


---


**`neemt`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | n | eemt | 3823 | 3 | 3 | 413
2 | ne | emt | 4170 | 14 | 14 | 81
3 | nee | mt | 1829 | 26 | 26 | 13
4 | neem | t | 4018 | 925 | 925 | 3

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
4 | neem | t | 9380 | 3 | 3 | 3470
3 | nee | mt | 653 | 10 | 10 | 47
2 | ne | emt | 298 | 47 | 47 | 22
1 | n | eemt | 245 | 156 | 156 | 3


---


**`aengeschreeven`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | a | engeschreeven | 9435 | 0 | 0 | 1114
2 | ae | ngeschreeven | 3053 | 0 | 0 | 260
3 | aen | geschreeven | 2796 | 1 | 1 | 230
4 | aeng | eschreeven | 354 | 2 | 2 | 92
5 | aenge | schreeven | 493 | 2 | 2 | 86

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
13 | aengeschreeve | n | 7997 | 0 | 0 | 6243
12 | aengeschreev | en | 13967 | 0 | 0 | 5556
11 | aengeschree | ven | 2360 | 0 | 0 | 205
10 | aengeschre | even | 2381 | 1 | 1 | 79
9 | aengeschr | eeven | 441 | 1 | 1 | 12


---


**`H`**

*no splits*


---


**`hunner`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | h | unner | 6945 | 0 | 0 | 571
2 | hu | nner | 2067 | 1 | 1 | 56
3 | hun | ner | 899 | 8 | 8 | 8
4 | hunn | er | 3343 | 496 | 496 | 2
5 | hunne | r | 1781 | 491 | 491 | 1

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
5 | hunne | r | 4102 | 2 | 2 | 1102
4 | hunn | er | 6594 | 2 | 2 | 850
3 | hun | ner | 107 | 5 | 5 | 11
2 | hu | nner | 116 | 29 | 29 | 1
1 | h | unner | 253 | 253 | 253 | 0


---


...

**`gelegentheyt`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | g | elegentheyt | 4884 | 1 | 1 | 2251
2 | ge | legentheyt | 7676 | 1 | 1 | 1780
3 | gel | egentheyt | 3689 | 5 | 5 | 160
4 | gele | gentheyt | 2664 | 5 | 5 | 43
5 | geleg | entheyt | 1971 | 6 | 6 | 17

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
11 | gelegenthey | t | 9380 | 1 | 1 | 3470
10 | gelegenthe | yt | 3418 | 2 | 2 | 402
9 | gelegenth | eyt | 1917 | 2 | 2 | 301
8 | gelegent | heyt | 616 | 3 | 3 | 204
7 | gelegen | theyt | 115 | 11 | 11 | 31


---


**`ongelegentheyt`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | o | ngelegentheyt | 5922 | 0 | 0 | 1204
2 | on | gelegentheyt | 1670 | 1 | 1 | 529
3 | ong | elegentheyt | 231 | 1 | 1 | 91
4 | onge | legentheyt | 357 | 1 | 1 | 89
5 | ongel | egentheyt | 627 | 5 | 5 | 15

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
13 | ongelegenthey | t | 9380 | 0 | 0 | 3470
12 | ongelegenthe | yt | 3418 | 0 | 0 | 402
11 | ongelegenth | eyt | 1917 | 0 | 0 | 301
10 | ongelegent | heyt | 616 | 1 | 1 | 204
9 | ongelegen | theyt | 115 | 2 | 2 | 31


---


**`heeft`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | h | eeft | 6945 | 7 | 7 | 571
2 | he | eft | 8825 | 16 | 16 | 166
3 | hee | ft | 4784 | 38 | 38 | 23
4 | heef | t | 1354 | 925 | 925 | 4

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
4 | heef | t | 9380 | 4 | 4 | 3470
3 | hee | ft | 570 | 16 | 16 | 64
2 | he | eft | 965 | 94 | 94 | 20
1 | h | eeft | 1376 | 253 | 253 | 7


---


**`gesanten`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | g | esanten | 4884 | 0 | 0 | 2251
2 | ge | santen | 7676 | 1 | 1 | 1780
3 | ges | anten | 1185 | 18 | 18 | 251
4 | gesa | nten | 554 | 25 | 25 | 19
5 | gesan | ten | 1666 | 320 | 320 | 5

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
7 | gesante | n | 7997 | 1 | 1 | 6243
6 | gesant | en | 13967 | 4 | 4 | 5556
5 | gesan | ten | 3766 | 4 | 4 | 656
4 | gesa | nten | 571 | 9 | 9 | 70
3 | ges | anten | 525 | 106 | 106 | 33


---


**`ootmoedige`**

pos | morf | main | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
1 | o | otmoedige | 5922 | 0 | 0 | 1204
2 | oo | tmoedige | 1677 | 0 | 0 | 75
3 | oot | moedige | 10 | 2 | 2 | 3
4 | ootm | oedige | 28 | 1 | 1 | 3
5 | ootmo | edige | 182 | 7 | 7 | 3

pos | main | morf | qual-morf | #-morfs | qual-main | #-main
--- | ---        | ---         | ---       | ---     | ---       | ---
9 | ootmoedig | e | 8676 | 0 | 0 | 4515
8 | ootmoedi | ge | 1751 | 1 | 1 | 449
7 | ootmoed | ige | 474 | 2 | 2 | 131
6 | ootmoe | dige | 405 | 2 | 2 | 40
5 | ootmo | edige | 935 | 2 | 2 | 13


---
